# Intro

This is a demo covering:
1. Setting up an Agent for Amazon Bedrock using AWS Console
2. How to debugg the Agent
3. How to deploy an Agent
4. How to call it from code

## Scenario we are trying to achieve
You have a task to setup an automated bot as a backend for your application to enable your users to book trips to the required destination by bus or by plane for a specific date. Users wants to communicate with a system in a natual language. System should be intelligent enough to gather missing information from the user, and automatically get this information from differen sources if required. When the trip is booked, agent need to find relevant information from internal documents about maximum baggage allowance and what will happen if the trip will be delayed, and include this information into the final response for the user.

In the same directory you will find next assets:
- `trip-terms` - folder with Markdown files with generated information for Flight terms and Bus terms. These are used in Knowledge base for RAG use-case.
- `user-details-api` - folder with Python code for AWS Lambda function which is called for getting user details based on the user id, and OpenAPI schema for this Lambda function, so Agent for Amazon Bedrock can use this API as an Action Group.
- `trip-booking-api` - folder with Python code for AWS Lambda function which is called for booking the trip based on provided information, and OpenAPI schema for this Lambda function, so Agent for Amazon Bedrock can use this API as an Action Group.

> **Warning**
> Be aware, it is NOT CDK or CFN project, so you will have to clean up all reasources manually!
> Be aware of costs assiciated with the solution, check out costs on https://calculator.aws/#/

# Setting up a Knowledge Base

## Upload trip terms to S3 bucket
Take Markdown files from `trip-terms/` folder and upload it to a new S3 bucket.

### Create bucket

In [ ]:
import boto3

# Create an STS client
sts_client = boto3.client('sts')

# Call the get_caller_identity method to retrieve account details
account_id = sts_client.get_caller_identity()["Account"]

# Format the bucket name
bucket_name_knowledge = f'knowledge-base-agents-demo-{account_id}'

# Create an S3 client
s3_client = boto3.client('s3')

# Check if the bucket already exists
existing_buckets = s3_client.list_buckets()

if any(bucket['Name'] == bucket_name_knowledge for bucket in existing_buckets['Buckets']):
    print(f'Bucket {bucket_name_knowledge} already exists.')
else:
    # Attempt to create the bucket if it does not exist
    try:
        s3_client.create_bucket(Bucket=bucket_name_knowledge)
        print(f'Bucket {bucket_name_knowledge} created successfully.')
    except Exception as e:
        print(f'Error creating bucket: {e}')


### Upload files

In [ ]:
import os

# Specify the directory containing the files
directory = './trip-terms'

# Iterate over the files in the directory
for filename in os.listdir(directory):
    # Construct the full file path
    file_path = os.path.join(directory, filename)
    
    # Check if it's a file (and not a directory/subdirectory)
    if os.path.isfile(file_path):
        # Define the S3 key (name) for the file
        # You might want to customize this to include subdirectories or modify file names
        s3_key = filename
        
        try:
            # Upload the file to S3
            s3_client.upload_file(Filename=file_path, Bucket=bucket_name, Key=s3_key)
            print(f'Successfully uploaded {filename} to {bucket_name}/{s3_key}.')
        except Exception as e:
            print(f'Error uploading {filename}: {e}')

### Setup Knowledge base

> NOTE - we will do it via Console, so it can create an OpenSearch cluster for us, otherwise you have to create an OpneSearch Cluster separately

Follow the documentation here - https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-create.html and use your bucket `knowledge-base-agents-demo-{YOUR-ACCOUNT-ID}` as input for it.

> NOTE - after you create it, you need to SYNC it, use button "SYNC" in "Data Source" section.

# Create Lambdas for Agent

### First, create a new bucket for OpenAPI schema

In [ ]:
# Format the bucket name
bucket_name_schema = f'openapi-schema-agents-demo-{account_id}'

# List all buckets and check if the bucket already exists
existing_buckets = s3_client.list_buckets()

if any(bucket['Name'] == bucket_name_schema for bucket in existing_buckets['Buckets']):
    print(f'Bucket {bucket_name_schema} already exists.')
else:
    # Create the bucket if it does not exist
    try:
        s3_client.create_bucket(Bucket=bucket_name_schema)
        print(f'Bucket {bucket_name_schema} created successfully.')
    except Exception as e:
        print(f'Error creating bucket: {e}')


### Upload Open API schemas

In [ ]:
# Define the files and their respective S3 keys
files_to_upload = {
    './user-details-api/user-details-open-api-schema.json': 'user-details-open-api-schema.json',
    './trip-booking-api/book-trip-open-api-schema.json': 'book-trip-open-api-schema.json'
}

# Upload each file to the specified S3 bucket
for local_path, s3_key in files_to_upload.items():
    try:
        s3_client.upload_file(Filename=local_path, Bucket=bucket_name_schema, Key=s3_key)
        print(f'Successfully uploaded {s3_key} to {bucket_name_schema}.')
    except Exception as e:
        print(f'Error uploading {s3_key}: {e}')


### Create lambdas

1st - Create Basic Lambda Execution Role

In [ ]:
import json

# Initialize the Boto3 client for IAM
iam_client = boto3.client('iam')

# The trust relationship policy document
trust_relationship = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# Policy document for CloudWatch Logs actions
policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "*"
        }
    ]
}

try:
    # Create the IAM role
    role_name = 'LambdaExecutionRole'
    description = 'Execution role for Lambda with CloudWatch logs access'

    create_role_response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_relationship),
        Description=description
    )
    role_arn = create_role_response['Role']['Arn']
    print(f'Created IAM role: {role_name}')

    # Create and attach the inline policy for CloudWatch Logs
    policy_name = 'LambdaCloudWatchLogsPolicy'
    iam_client.put_role_policy(
        RoleName=role_name,
        PolicyName=policy_name,
        PolicyDocument=json.dumps(policy_document)
    )
    print(f'Attached inline policy for CloudWatch Logs to {role_name}')

    print(f'Role ARN: {role_arn}')

except Exception as e:
    print(f'Error creating IAM role or attaching policy: {e}')


2nd - Create Lambda for User details API

In [ ]:
import zipfile
import os

# Package the Python script into a ZIP file
zip_file_path = 'user-details.zip'
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    zipf.write('./user-details-api/user-details.py', 'user-details.py')

# Initialize the Boto3 client for Lambda
lambda_client = boto3.client('lambda')

# Specify the IAM role ARN (replace this with your actual IAM role ARN)
role_arn = f'arn:aws:iam::{account_id}:role/LambdaExecutionRole'

# Create the Lambda function
try:
    with open(zip_file_path, 'rb') as zipf:
        response = lambda_client.create_function(
            FunctionName='UserDetailsFunction',
            Runtime='python3.8',  # Specify the correct runtime according to your Python version
            Role=role_arn,
            Handler='user-details.lambda_handler',  # Format: <file-name-without-extension>.<handler-function-name>
            Code={
                'ZipFile': zipf.read(),
            },
            Description='A Lambda function to handle user details.',
            Timeout=15,  # Maximum execution time in seconds (adjust as needed)
            MemorySize=128,  # Allocated memory in MB (adjust as needed)
        )
    print("Lambda function created successfully:", response)
except Exception as e:
    print("Error creating Lambda function:", e)
finally:
    # Clean up: Remove the ZIP file after the Lambda function has been created
    os.remove(zip_file_path)


3rd - Create Lambda for Trip Booking API

In [ ]:
import zipfile
import os

# Package the Python script into a ZIP file
zip_file_path = 'trip-booking.zip'
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    zipf.write('./trip-booking-api/trip-booking-lambda.py', 'trip-booking-lambda.py')

# Initialize the Boto3 client for Lambda
lambda_client = boto3.client('lambda')

# Specify the IAM role ARN (replace this with your actual IAM role ARN)
role_arn = f'arn:aws:iam::{account_id}:role/LambdaExecutionRole'

# Create the Lambda function
try:
    with open(zip_file_path, 'rb') as zipf:
        response = lambda_client.create_function(
            FunctionName='TripBookingFunction',
            Runtime='python3.8',  # Specify the correct runtime according to your Python version
            Role=role_arn,
            Handler='user-details.lambda_handler',  # Format: <file-name-without-extension>.<handler-function-name>
            Code={
                'ZipFile': zipf.read(),
            },
            Description='A Lambda function to handle trip bookings.',
            Timeout=15,  # Maximum execution time in seconds (adjust as needed)
            MemorySize=128,  # Allocated memory in MB (adjust as needed)
        )
    print("Lambda function created successfully:", response)
except Exception as e:
    print("Error creating Lambda function:", e)
finally:
    # Clean up: Remove the ZIP file after the Lambda function has been created
    os.remove(zip_file_path)


# Create Agent

Now, let's create an agent it-self.

TBD - follow documentation on https://docs.aws.amazon.com/bedrock/latest/userguide/agents-create.html 

Use the next Prompt for the Basic Instructions for Agent:
> You are an agent who performs the next tasks for the user:
> 1/ Get the information about the user based on provided userId, you obtain his name and surname by that.
> 2/ Do a booking for a trip for his name and surname you found.
> 3/ In confirmation message include information about baggage size allowance and what will happen in case of trip delay for selected type of transport.

I recommend to use Claude V2 as an LLM behind Agent, as it understand instructions better. The trade-off is the speed (Claude Instant is faster) and price (Claude Instant is cheaper). So, if you need to optimize for performance and cost - try out Claude Instant, but you need to test if it will work properly with provided Basic instructions and most probably it will have to be adjusted.

Then, define you Action Group. Here you need to define your Lambdas created in previous steps and define their Open API schemas uploaded to S3 bucket `openapi-schema-agents-demo-{YOUR-ACCOUNT-ID}`. In total, you will have 2 Action Groups: 1/ for User Details API, 2/ for Trip Booking API.

Then, define a Knowledge base you created before with `trip-terms`. For Description use something like `Here is information about general terms for trips by bus and airplane.`. It will be used by your Agent to undestand what it can find there.

> **Warning**
> AFTER you will create Agent, you need to add Resource Based Permissions for Lambda, so it can be called by Bedrock service! Here is an example of such policy. Do this for each Action Group / Lambda.


# Test your Agent and see the Traces to Debugg its behaviour

To do that, follow docs. https://docs.aws.amazon.com/bedrock/latest/userguide/agents-test.html.

If you will open your agent, on the left side you will see the chat window, and if you will expand it, while you will be sending requests in the chat, you will see Traces in the right menu. These are Chain of Thoughts and actions performed by your agent to accomplish the task.

Try to put into the chat:
> I have a userid 1, do a booking of trip to Bratislava at 12 Feb 2024 by bus.

# Deploy your agent

Follow docs - https://docs.aws.amazon.com/bedrock/latest/userguide/agents-deploy.html

You will create an Alias an Version of your Agent, and then associate your Version with an Alias. E.g. have Alias `PROD` and Version `Version 1`.

# Call your agent from the code via Python SDK 

You can do the same in other languages using AWS SDK.

### First, environment setup

In [ ]:
import boto3
import uuid
import json

client = boto3.client('bedrock-agent-runtime')

### Define your Agent ID with Alias ID and Prompt

In [ ]:
agent_id = '1RYL9BCNPV'
agent_alias_id = 'QPC7UHH6KF'
session_id = str(uuid.uuid4().hex)

In [ ]:
user_prompt = 'I have a userid 2, do a booking of trip to Paris at 24 may 2024 by plane.'

### Invoke Bedrock client

You invoke your Bedrock client based on provided agent id and alias id, pass it your user prompt, and then parse the result. Optionally you can pass chat history, but this is not included into the demo.

You can also parse and see Traces as well (similar as in the AWS Console). I included it here, so you can see Traces as well in real-time.

In [ ]:
response = client.invoke_agent(
    inputText = user_prompt,
    agentId = agent_id,
    agentAliasId = agent_alias_id,
    sessionId = session_id,
    enableTrace = True
)

for event in response['completion']:
    if 'chunk' in event:
        data = event['chunk']['bytes']
        answer = data.decode('utf8')
        print(f'Answer:\n{answer}')
    elif 'trace' in event:
        print(json.dumps(event['trace'], indent=2))
